In [4]:
import os
os.chdir('../')
%pwd

'/Users/vishalsharma/Downloads/Coccidiosis Chicken Disease Classifier'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_dir: Path

In [11]:
from src.coccidiosis_chicken_disease_classifier.constants import *
from src.coccidiosis_chicken_disease_classifier.utils.common import read_yaml, create_directories


class PrepareCallbacksConfigManager:
    def __init__(self, config_file_path: Path = CONFIG_FILE_PATH, parmas_file_path: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(parmas_file_path)
        
        create_directories(self.config.artifacts_root)

    def get_prepare_callbacks(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(model_ckpt_dir)
        create_directories(config.tensorboard_root_log_dir)

        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_dir=Path(config.checkpoint_model_filepath))
    



In [12]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_log_dir = os.path.join(self.config.tensorboard_log_dir, f"tb_logs_{timestamp}")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)
        return tensorboard_callback
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_dir,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]




In [13]:
try:
    config_obj = PrepareCallbacksConfigManager(config_file_path=CONFIG_FILE_PATH, parmas_file_path=PARAMS_FILE_PATH)
    prepare_callbacks_config = config_obj.get_prepare_callbacks()
    callbacks_obj = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks = callbacks_obj.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

2025-12-06 17:39:37,781 - INFO - common:21 - YAML file: config/config.yaml loaded successfully
2025-12-06 17:39:37,783 - INFO - common:21 - YAML file: params.yaml loaded successfully
2025-12-06 17:39:37,784 - INFO - common:34 - Created directory at: artifacts
2025-12-06 17:39:37,784 - INFO - common:34 - Created directory at: artifacts/prepare_callbacks/checkpoints
2025-12-06 17:39:37,785 - INFO - common:34 - Created directory at: artifacts/prepare_callbacks/tensorboard_logs
